In [ ]:
%pip install requests
%pip install azure-ai-vision-imageanalysis

In [3]:
import requests
from azure.ai.vision.imageanalysis import ImageAnalysisClient
from azure.ai.vision.imageanalysis.models import VisualFeatures
from azure.core.credentials import AzureKeyCredential

## 객체 감지

In [1]:
import requests
import gradio as gr
from PIL import Image, ImageDraw, ImageFont

def request_vision(image_path):
    features = "SmartCrops"
    endpoint = f"https://fimtrus-computervision2.cognitiveservices.azure.com/computervision/imageanalysis:analyze"
    api_key = "ee6d3d3de333480291b572fc7147b015"

    headers = {
        "Content-Type": "application/octet-stream",
        "Ocp-Apim-Subscription-Key": api_key
    }
    params ={
        "api-version": "2024-02-01",
        "features": features,
        "smartcrops-aspect-ratios": "1,1.1,1.5"
    }

    with open(image_path, 'rb') as image:
        image_data = image.read()

    response = requests.post(endpoint, 
                             headers=headers,
                             params=params,
                             data=image_data)

    response_json = response.json()
    print(response_json)

    return response_json

def change_image(image_path):

    if image_path is None:
        return None
    
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)
    try:
        font = ImageFont.truetype("NanumGothic.ttf", 30)
    except IOError:
        # Arial 폰트가 없으면 기본 폰트를 사용합니다.
        font = ImageFont.load_default()
    
    # request_vision을 통해 데이터를 받아 온다.
    # 받은 데이터를 파싱해서 image에 그린다.

    response_json = request_vision(image_path)
    # 받은 데이터를 파싱해서 image에 그린다.  
    if "objectsResult" in response_json and "values" in response_json["objectsResult"]:  
        for obj in response_json["objectsResult"]["values"]:  
            box = obj["boundingBox"]  
            x, y, w, h = box["x"], box["y"], box["w"], box["h"]  
            print(x, y, w, h)
            draw.rectangle([(x, y), (x + w, y + h)], outline="red", width=2)  
            if "tags" in obj:  
                for tag in obj["tags"]:  
                    draw.text((x, y - 10), tag["name"], fill="red", font=font)  

    if "tagsResult" in response_json and "values" in response_json["tagsResult"]:  
        for obj in response_json["tagsResult"]["values"]:  
            print(obj['name'], int(obj['confidence']*100))

    if "smartCropsResult" in response_json and "values" in response_json["smartCropsResult"]:  
        for obj in response_json["smartCropsResult"]["values"]:  
            box = obj["boundingBox"]  
            x, y, w, h = box["x"], box["y"], box["w"], box["h"]  
            print(x, y, w, h)
            draw.rectangle([(x, y), (x + w, y + h)], outline="red", width=2)  
            draw.text((x, y - 10), str(obj['aspectRatio']), fill="red", font=font)  


    return image

with gr.Blocks() as demo:

    input_image = gr.Image(label="이미지 선택", type="filepath", width=800)
    output_image = gr.Image(label="출력 이미지", type="pil", width=800, interactive=False)

    input_image.change(fn=change_image, inputs=[input_image], outputs=[output_image])

demo.launch()

/Users/fimtrus/.pyenv/versions/3.11.9/envs/openai/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


{'modelVersion': '2023-10-01', 'metadata': {'width': 400, 'height': 300}, 'smartCropsResult': {'values': [{'aspectRatio': 1.0, 'boundingBox': {'x': 68, 'y': 0, 'w': 299, 'h': 299}}, {'aspectRatio': 1.1, 'boundingBox': {'x': 68, 'y': 0, 'w': 302, 'h': 274}}, {'aspectRatio': 1.5, 'boundingBox': {'x': 41, 'y': 21, 'w': 358, 'h': 238}}]}}
68 0 299 299
68 0 302 274
41 21 358 238


## 배경 제거

In [9]:
import requests
import gradio as gr
import io

def request_background_removal(image_path):
    endpoint = "https://fimtrus-computervision2.cognitiveservices.azure.com/computervision/imageanalysis:segment"
    api_key = "ee6d3d3de333480291b572fc7147b015"

    headers = {
        "Content-Type": "application/octet-stream",
        "Ocp-Apim-Subscription-Key": api_key
    }

    params = {
        "api-version": "2023-02-01-preview",
        "mode": "backgroundRemoval"
    }

    with open(image_path, 'rb') as image:
        image_data = image.read()

    response = requests.post(endpoint,
                             headers=headers,
                             params=params,
                             data=image_data)
    
    print(response.content)
    return response.content

# request_background_removal('/Users/fimtrus/Pictures/002.다이빙/501be83cf4a806200d4ab58d39ba71db_thumbnail.JPG')

def change_image2(image_path):
    image_data = request_background_removal(image_path=image_path)

    image = Image.open(io.BytesIO(image_data))
    return image

with gr.Blocks() as demo2:

    input_image = gr.Image(type="filepath")
    output_image = gr.Image(type="pil")

    input_image.change(fn=change_image2, inputs=[input_image], outputs=[output_image])

demo2.launch()



Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x90\x00\x00\x01,\x08\x06\x00\x00\x00\xed\xb7\xe5\xc2\x00\x00\x00\x04gAMA\x00\x00\xb1\x8f\x0b\xfca\x05\x00\x00\x00\x01sRGB\x00\xae\xce\x1c\xe9\x00\x00\x00 cHRM\x00\x00z&\x00\x00\x80\x84\x00\x00\xfa\x00\x00\x00\x80\xe8\x00\x00u0\x00\x00\xea`\x00\x00:\x98\x00\x00\x17p\x9c\xbaQ<\x00\x00 \x00IDATx\x9c\xec\xbd\xdd\x92$G\x96\xdf\xf7;\xee\x1e\x91\x99U\x8d\xc6b1\xb3\x9c\x15\x97\\\xa3\x91\x17\x14\xafh\x94I7z\x00\xc9d\xc6\xe7\x11\x9fB&]\xc9L\x97\xa2\x9e\x85\xa6+^\x90\xb4%9\\\xd9~\xcd\xec\xec\xec\x003@wWeF\xb8\xfb9\xba8\xee\x91\x91Q\x99U\r\xf4`\x00p\xe1m\xd5\x91\x99\x11\xe1\xe1\xe1\xe1q>\xfe\xe7K\xfe\xcf\xff\xeb_#"\x00\x88\x08!\x84\'\xdf\xfb\xe7\xf5\xef\xfeY\x89\x08\x88\xde\xd8\x0f)\xa5\x8b\xdf\xd6\xc7\x00\x98\xd9r\x9d\xed_\xff=\xc6x1\xaeu\x7f\xf1\xca5\xd7\x9fc\x8c\x17\xd7^\xf7\xdb\xf7\xf7q\x00\xa8*\xaa\x8a\x99-\xbfm\xb7\xeb\xeb\xa8\xea\xc5\x98\x80\xe5\x1a"B\x8c\x11U\xe5t:\xf1\xee\xdd;\x8e\xc7#\xa7\xd3\x89\xc7\xc7G\x1e\x8eG\xf6\xaf?\xe1\xd5\xeb\x8f